In [1]:
import pandas as pd
import numpy as np
import catboost as cb
from sklearn.model_selection import KFold
from itertools import product,chain
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from lightgbm import LGBMClassifier

In [2]:
# read in the train and test data from csv files

train_set = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [3]:
train_set.head()

,Applicant_ID,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,...,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50,default_status
0,Apcnt_1000000,3436.0,0.28505,1.6560,0.0,0.000,0.0,10689720.0,252072.0,4272776.0,...,0.392854,2.02,0.711632,0.0,0.0,charge,NaN,1.129518,0.044335,no
1,Apcnt_1000004,3456.0,0.67400,0.2342,0.0,0.000,0.0,898979.0,497531.0,9073814.0,...,0.314281,8.08,0.183584,NaN,0.0,charge,349.805730,1.620483,0.322436,no
2,Apcnt_1000008,3276.0,0.53845,3.1510,0.0,6.282,NaN,956940.0,NaN,192944.0,...,0.162965,18.18,0.791136,0.0,0.0,charge,NaN,1.513370,0.011640,yes
3,Apcnt_1000012,3372.0,0.17005,0.5050,0.0,0.000,192166.0,3044703.0,385499.0,3986472.0,...,0.488884,2.02,0.685168,NaN,0.0,charge,89.940100,0.664452,0.082729,no
4,Apcnt_1000016,3370.0,0.77270,1.1010,0.0,0.000,1556.0,214728.0,214728.0,1284089.0,...,0.275000,12.12,0.438168,0.0,0.0,charge,97.887502,1.427891,0.045630,no


In [4]:
test.head()

,Applicant_ID,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,...,form_field41,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50
0,Apcnt_1000032,3236.0,0.34875,10.2006,0.0000,0.0,418564.0,418564.0,418564.0,540710.0,...,NaN,0.825000,1.01,0.800000,NaN,0.0,charge,NaN,0.000000,0.011221
1,Apcnt_1000048,3284.0,1.27360,2.9606,9.0198,0.0,0.0,9858816.0,49014.0,1510098.0,...,18.8415,0.507694,4.04,0.623248,1.0,0.0,lending,NaN,0.504974,0.043525
2,Apcnt_1000052,NaN,0.27505,0.0600,0.0000,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,0.00,NaN,NaN,NaN,charge,NaN,0.000000,NaN
3,Apcnt_1000076,3232.0,0.28505,2.8032,0.0000,0.0,0.0,473802.0,473802.0,1724437.0,...,NaN,0.916663,2.02,0.464224,NaN,NaN,charge,90.163742,0.788809,0.104029
4,Apcnt_1000080,3466.0,2.09545,0.8318,2.5182,0.0,19839.0,1150662.0,1150662.0,7860523.0,...,NaN,0.234047,23.23,0.726688,0.0,0.0,lending,1303.587148,1.637733,0.163124


In [5]:
train_set.isnull().sum()

Applicant_ID          0
form_field1        2529
form_field2        3844
form_field3         355
form_field4         355
form_field5         355
form_field6       13360
form_field7        5163
form_field8       13360
form_field9        8008
form_field10        355
form_field11      31421
form_field12       9895
form_field13       5889
form_field14          0
form_field15      22475
form_field16      13036
form_field17      11151
form_field18      10402
form_field19          4
form_field20        355
form_field21      15854
form_field22      20400
form_field23      28123
form_field24      13297
form_field25       5450
form_field26       7438
form_field27       9299
form_field28        355
form_field29        355
form_field30      25509
form_field31      39408
form_field32       5450
form_field33       1256
form_field34        355
form_field35      23148
form_field36       1995
form_field37       5450
form_field38        355
form_field39       4211
form_field40      43729
form_field41    

In [6]:
test.isnull().sum()

Applicant_ID        0
form_field1      1110
form_field2      1709
form_field3       146
form_field4       146
form_field5       146
form_field6      5604
form_field7      2231
form_field8      5604
form_field9      3400
form_field10      147
form_field11    13398
form_field12     4183
form_field13     2463
form_field14        0
form_field15     9592
form_field16     5474
form_field17     4695
form_field18     4369
form_field19        0
form_field20      147
form_field21     6707
form_field22     8724
form_field23    12125
form_field24     5605
form_field25     2256
form_field26     3172
form_field27     3910
form_field28      147
form_field29      147
form_field30    10908
form_field31    16810
form_field32     2256
form_field33      495
form_field34      147
form_field35     9866
form_field36      903
form_field37     2256
form_field38      147
form_field39     1829
form_field40    18828
form_field41    16349
form_field42      578
form_field43      250
form_field44     2362
form_field

In [7]:
train_set = train_set.fillna(-9999)
test = test.fillna(-9999)

In [8]:
train_set.head()

,Applicant_ID,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,...,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50,default_status
0,Apcnt_1000000,3436.0,0.28505,1.6560,0.0,0.000,0.0,10689720.0,252072.0,4272776.0,...,0.392854,2.02,0.711632,0.0,0.0,charge,-9999.000000,1.129518,0.044335,no
1,Apcnt_1000004,3456.0,0.67400,0.2342,0.0,0.000,0.0,898979.0,497531.0,9073814.0,...,0.314281,8.08,0.183584,-9999.0,0.0,charge,349.805730,1.620483,0.322436,no
2,Apcnt_1000008,3276.0,0.53845,3.1510,0.0,6.282,-9999.0,956940.0,-9999.0,192944.0,...,0.162965,18.18,0.791136,0.0,0.0,charge,-9999.000000,1.513370,0.011640,yes
3,Apcnt_1000012,3372.0,0.17005,0.5050,0.0,0.000,192166.0,3044703.0,385499.0,3986472.0,...,0.488884,2.02,0.685168,-9999.0,0.0,charge,89.940100,0.664452,0.082729,no
4,Apcnt_1000016,3370.0,0.77270,1.1010,0.0,0.000,1556.0,214728.0,214728.0,1284089.0,...,0.275000,12.12,0.438168,0.0,0.0,charge,97.887502,1.427891,0.045630,no


In [9]:
test.head()

,Applicant_ID,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,...,form_field41,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50
0,Apcnt_1000032,3236.0,0.34875,10.2006,0.0000,0.0,418564.0,418564.0,418564.0,540710.0,...,-9999.0000,0.825000,1.01,0.800000,-9999.0,0.0,charge,-9999.000000,0.000000,0.011221
1,Apcnt_1000048,3284.0,1.27360,2.9606,9.0198,0.0,0.0,9858816.0,49014.0,1510098.0,...,18.8415,0.507694,4.04,0.623248,1.0,0.0,lending,-9999.000000,0.504974,0.043525
2,Apcnt_1000052,-9999.0,0.27505,0.0600,0.0000,0.0,-9999.0,-9999.0,-9999.0,-9999.0,...,-9999.0000,-9999.000000,0.00,-9999.000000,-9999.0,-9999.0,charge,-9999.000000,0.000000,-9999.000000
3,Apcnt_1000076,3232.0,0.28505,2.8032,0.0000,0.0,0.0,473802.0,473802.0,1724437.0,...,-9999.0000,0.916663,2.02,0.464224,-9999.0,-9999.0,charge,90.163742,0.788809,0.104029
4,Apcnt_1000080,3466.0,2.09545,0.8318,2.5182,0.0,19839.0,1150662.0,1150662.0,7860523.0,...,-9999.0000,0.234047,23.23,0.726688,0.0,0.0,lending,1303.587148,1.637733,0.163124


In [10]:
train_set.isnull().sum()

Applicant_ID      0
form_field1       0
form_field2       0
form_field3       0
form_field4       0
form_field5       0
form_field6       0
form_field7       0
form_field8       0
form_field9       0
form_field10      0
form_field11      0
form_field12      0
form_field13      0
form_field14      0
form_field15      0
form_field16      0
form_field17      0
form_field18      0
form_field19      0
form_field20      0
form_field21      0
form_field22      0
form_field23      0
form_field24      0
form_field25      0
form_field26      0
form_field27      0
form_field28      0
form_field29      0
form_field30      0
form_field31      0
form_field32      0
form_field33      0
form_field34      0
form_field35      0
form_field36      0
form_field37      0
form_field38      0
form_field39      0
form_field40      0
form_field41      0
form_field42      0
form_field43      0
form_field44      0
form_field45      0
form_field46      0
form_field47      0
form_field48      0
form_field49      0


In [11]:
test.isnull().sum()

Applicant_ID    0
form_field1     0
form_field2     0
form_field3     0
form_field4     0
form_field5     0
form_field6     0
form_field7     0
form_field8     0
form_field9     0
form_field10    0
form_field11    0
form_field12    0
form_field13    0
form_field14    0
form_field15    0
form_field16    0
form_field17    0
form_field18    0
form_field19    0
form_field20    0
form_field21    0
form_field22    0
form_field23    0
form_field24    0
form_field25    0
form_field26    0
form_field27    0
form_field28    0
form_field29    0
form_field30    0
form_field31    0
form_field32    0
form_field33    0
form_field34    0
form_field35    0
form_field36    0
form_field37    0
form_field38    0
form_field39    0
form_field40    0
form_field41    0
form_field42    0
form_field43    0
form_field44    0
form_field45    0
form_field46    0
form_field47    0
form_field48    0
form_field49    0
form_field50    0
dtype: int64

In [12]:
# convert categorical columns to integers
category_cols = ['form_field47', 'default_status']
for column in category_cols:
    train_set[column] = train_set[column].astype('category').cat.codes
    test['form_field47'] = test['form_field47'].astype('category').cat.codes

In [13]:
train_set.head()

,Applicant_ID,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,...,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50,default_status
0,Apcnt_1000000,3436.0,0.28505,1.6560,0.0,0.000,0.0,10689720.0,252072.0,4272776.0,...,0.392854,2.02,0.711632,0.0,0.0,0,-9999.000000,1.129518,0.044335,0
1,Apcnt_1000004,3456.0,0.67400,0.2342,0.0,0.000,0.0,898979.0,497531.0,9073814.0,...,0.314281,8.08,0.183584,-9999.0,0.0,0,349.805730,1.620483,0.322436,0
2,Apcnt_1000008,3276.0,0.53845,3.1510,0.0,6.282,-9999.0,956940.0,-9999.0,192944.0,...,0.162965,18.18,0.791136,0.0,0.0,0,-9999.000000,1.513370,0.011640,1
3,Apcnt_1000012,3372.0,0.17005,0.5050,0.0,0.000,192166.0,3044703.0,385499.0,3986472.0,...,0.488884,2.02,0.685168,-9999.0,0.0,0,89.940100,0.664452,0.082729,0
4,Apcnt_1000016,3370.0,0.77270,1.1010,0.0,0.000,1556.0,214728.0,214728.0,1284089.0,...,0.275000,12.12,0.438168,0.0,0.0,0,97.887502,1.427891,0.045630,0


In [14]:
# split labels out of data sets    
train_label = train_set['default_status']
train_set = train_set.drop(['Applicant_ID','default_status'], axis=1) # remove labels
test_set = test.drop('Applicant_ID', axis=1) # dropping the first column because it has unique elements for all entries

In [15]:
x_train, x_test, y_train, y_test = train_test_split(train_set, train_label, test_size = 0.3)

In [16]:
clf = cb.CatBoostClassifier(n_estimators = 3000, early_stopping_rounds = 50, learning_rate = 0.01, random_seed =50)

In [17]:
# train default classifier    
clf.fit(x_train, y_train)

0:	learn: 0.6863184	total: 335ms	remaining: 16m 43s
1:	learn: 0.6800239	total: 472ms	remaining: 11m 47s
2:	learn: 0.6738700	total: 649ms	remaining: 10m 48s
3:	learn: 0.6676383	total: 809ms	remaining: 10m 6s
4:	learn: 0.6616215	total: 1s	remaining: 10m 1s
5:	learn: 0.6561957	total: 1.15s	remaining: 9m 35s
6:	learn: 0.6504395	total: 1.41s	remaining: 10m 1s
7:	learn: 0.6449189	total: 1.51s	remaining: 9m 23s
8:	learn: 0.6393798	total: 1.79s	remaining: 9m 55s
9:	learn: 0.6340389	total: 2.08s	remaining: 10m 22s
10:	learn: 0.6292903	total: 2.2s	remaining: 9m 56s
11:	learn: 0.6244351	total: 2.38s	remaining: 9m 51s
12:	learn: 0.6199433	total: 2.51s	remaining: 9m 37s
13:	learn: 0.6152649	total: 2.81s	remaining: 9m 58s
14:	learn: 0.6108678	total: 2.96s	remaining: 9m 49s
15:	learn: 0.6065708	total: 3.15s	remaining: 9m 46s
16:	learn: 0.6021529	total: 3.25s	remaining: 9m 29s
17:	learn: 0.5980694	total: 3.43s	remaining: 9m 28s
18:	learn: 0.5944743	total: 3.66s	remaining: 9m 34s
19:	learn: 0.5905840	t

160:	learn: 0.4284552	total: 28.5s	remaining: 8m 22s
161:	learn: 0.4282245	total: 28.6s	remaining: 8m 20s
162:	learn: 0.4279697	total: 28.7s	remaining: 8m 18s
163:	learn: 0.4277008	total: 28.9s	remaining: 8m 19s
164:	learn: 0.4274859	total: 29s	remaining: 8m 18s
165:	learn: 0.4272837	total: 29.2s	remaining: 8m 18s
166:	learn: 0.4270618	total: 29.3s	remaining: 8m 17s
167:	learn: 0.4267860	total: 29.4s	remaining: 8m 16s
168:	learn: 0.4265165	total: 29.6s	remaining: 8m 15s
169:	learn: 0.4262641	total: 29.7s	remaining: 8m 14s
170:	learn: 0.4260669	total: 29.9s	remaining: 8m 14s
171:	learn: 0.4258105	total: 30s	remaining: 8m 14s
172:	learn: 0.4255856	total: 30.3s	remaining: 8m 15s
173:	learn: 0.4253784	total: 30.4s	remaining: 8m 14s
174:	learn: 0.4251635	total: 30.7s	remaining: 8m 15s
175:	learn: 0.4249817	total: 30.8s	remaining: 8m 14s
176:	learn: 0.4247489	total: 31.1s	remaining: 8m 15s
177:	learn: 0.4245195	total: 31.2s	remaining: 8m 14s
178:	learn: 0.4243312	total: 31.4s	remaining: 8m 1

317:	learn: 0.4092388	total: 53.6s	remaining: 7m 32s
318:	learn: 0.4091781	total: 53.8s	remaining: 7m 32s
319:	learn: 0.4091195	total: 54s	remaining: 7m 32s
320:	learn: 0.4090642	total: 54.1s	remaining: 7m 31s
321:	learn: 0.4089995	total: 54.4s	remaining: 7m 32s
322:	learn: 0.4089426	total: 54.5s	remaining: 7m 31s
323:	learn: 0.4088761	total: 54.7s	remaining: 7m 31s
324:	learn: 0.4088237	total: 54.8s	remaining: 7m 31s
325:	learn: 0.4087647	total: 55.1s	remaining: 7m 31s
326:	learn: 0.4087132	total: 55.2s	remaining: 7m 31s
327:	learn: 0.4086545	total: 55.3s	remaining: 7m 30s
328:	learn: 0.4086034	total: 55.5s	remaining: 7m 30s
329:	learn: 0.4085446	total: 55.7s	remaining: 7m 30s
330:	learn: 0.4084927	total: 55.9s	remaining: 7m 30s
331:	learn: 0.4084371	total: 56.1s	remaining: 7m 30s
332:	learn: 0.4083829	total: 56.2s	remaining: 7m 30s
333:	learn: 0.4083161	total: 56.4s	remaining: 7m 30s
334:	learn: 0.4082704	total: 56.6s	remaining: 7m 29s
335:	learn: 0.4082231	total: 56.8s	remaining: 7m

471:	learn: 0.4021912	total: 1m 20s	remaining: 7m 11s
472:	learn: 0.4021485	total: 1m 20s	remaining: 7m 11s
473:	learn: 0.4021146	total: 1m 20s	remaining: 7m 10s
474:	learn: 0.4020809	total: 1m 21s	remaining: 7m 11s
475:	learn: 0.4020410	total: 1m 21s	remaining: 7m 10s
476:	learn: 0.4019989	total: 1m 21s	remaining: 7m 10s
477:	learn: 0.4019607	total: 1m 21s	remaining: 7m 10s
478:	learn: 0.4019202	total: 1m 22s	remaining: 7m 11s
479:	learn: 0.4018842	total: 1m 22s	remaining: 7m 11s
480:	learn: 0.4018412	total: 1m 22s	remaining: 7m 11s
481:	learn: 0.4017943	total: 1m 22s	remaining: 7m 11s
482:	learn: 0.4017596	total: 1m 22s	remaining: 7m 11s
483:	learn: 0.4017292	total: 1m 22s	remaining: 7m 11s
484:	learn: 0.4017016	total: 1m 23s	remaining: 7m 11s
485:	learn: 0.4016794	total: 1m 23s	remaining: 7m 11s
486:	learn: 0.4016441	total: 1m 23s	remaining: 7m 10s
487:	learn: 0.4016201	total: 1m 23s	remaining: 7m 10s
488:	learn: 0.4015855	total: 1m 23s	remaining: 7m 10s
489:	learn: 0.4015575	total:

624:	learn: 0.3975358	total: 1m 46s	remaining: 6m 45s
625:	learn: 0.3975031	total: 1m 47s	remaining: 6m 45s
626:	learn: 0.3974749	total: 1m 47s	remaining: 6m 45s
627:	learn: 0.3974507	total: 1m 47s	remaining: 6m 45s
628:	learn: 0.3974292	total: 1m 47s	remaining: 6m 45s
629:	learn: 0.3974094	total: 1m 47s	remaining: 6m 45s
630:	learn: 0.3973833	total: 1m 47s	remaining: 6m 45s
631:	learn: 0.3973419	total: 1m 48s	remaining: 6m 45s
632:	learn: 0.3973148	total: 1m 48s	remaining: 6m 45s
633:	learn: 0.3972914	total: 1m 48s	remaining: 6m 44s
634:	learn: 0.3972636	total: 1m 48s	remaining: 6m 44s
635:	learn: 0.3972373	total: 1m 48s	remaining: 6m 44s
636:	learn: 0.3972181	total: 1m 49s	remaining: 6m 44s
637:	learn: 0.3971893	total: 1m 49s	remaining: 6m 44s
638:	learn: 0.3971722	total: 1m 49s	remaining: 6m 44s
639:	learn: 0.3971436	total: 1m 49s	remaining: 6m 43s
640:	learn: 0.3971145	total: 1m 49s	remaining: 6m 43s
641:	learn: 0.3970959	total: 1m 49s	remaining: 6m 43s
642:	learn: 0.3970538	total:

...

2293:	learn: 0.3598598	total: 6m 25s	remaining: 1m 58s
2294:	learn: 0.3598309	total: 6m 25s	remaining: 1m 58s
2295:	learn: 0.3598142	total: 6m 25s	remaining: 1m 58s
2296:	learn: 0.3597886	total: 6m 25s	remaining: 1m 58s
2297:	learn: 0.3597634	total: 6m 25s	remaining: 1m 57s
2298:	learn: 0.3597439	total: 6m 26s	remaining: 1m 57s
2299:	learn: 0.3597172	total: 6m 26s	remaining: 1m 57s
2300:	learn: 0.3596898	total: 6m 26s	remaining: 1m 57s
2301:	learn: 0.3596678	total: 6m 26s	remaining: 1m 57s
2302:	learn: 0.3596506	total: 6m 26s	remaining: 1m 57s
2303:	learn: 0.3596278	total: 6m 27s	remaining: 1m 56s
2304:	learn: 0.3596250	total: 6m 27s	remaining: 1m 56s
2305:	learn: 0.3596087	total: 6m 27s	remaining: 1m 56s
2306:	learn: 0.3595943	total: 6m 27s	remaining: 1m 56s
2307:	learn: 0.3595757	total: 6m 27s	remaining: 1m 56s
2308:	learn: 0.3595532	total: 6m 27s	remaining: 1m 56s
2309:	learn: 0.3595343	total: 6m 27s	remaining: 1m 55s
2310:	learn: 0.3595173	total: 6m 27s	remaining: 1m 55s
2311:	lear

2443:	learn: 0.3569792	total: 6m 48s	remaining: 1m 32s
2444:	learn: 0.3569656	total: 6m 48s	remaining: 1m 32s
2445:	learn: 0.3569482	total: 6m 48s	remaining: 1m 32s
2446:	learn: 0.3569323	total: 6m 49s	remaining: 1m 32s
2447:	learn: 0.3569290	total: 6m 49s	remaining: 1m 32s
2448:	learn: 0.3569100	total: 6m 49s	remaining: 1m 32s
2449:	learn: 0.3568879	total: 6m 49s	remaining: 1m 31s
2450:	learn: 0.3568650	total: 6m 49s	remaining: 1m 31s
2451:	learn: 0.3568424	total: 6m 49s	remaining: 1m 31s
2452:	learn: 0.3568218	total: 6m 49s	remaining: 1m 31s
2453:	learn: 0.3567998	total: 6m 50s	remaining: 1m 31s
2454:	learn: 0.3567857	total: 6m 50s	remaining: 1m 31s
2455:	learn: 0.3567620	total: 6m 50s	remaining: 1m 30s
2456:	learn: 0.3567445	total: 6m 50s	remaining: 1m 30s
2457:	learn: 0.3567219	total: 6m 50s	remaining: 1m 30s
2458:	learn: 0.3567048	total: 6m 50s	remaining: 1m 30s
2459:	learn: 0.3566839	total: 6m 50s	remaining: 1m 30s
2460:	learn: 0.3566586	total: 6m 50s	remaining: 1m 30s
2461:	lear

2594:	learn: 0.3541045	total: 7m 10s	remaining: 1m 7s
2595:	learn: 0.3540832	total: 7m 11s	remaining: 1m 7s
2596:	learn: 0.3540656	total: 7m 11s	remaining: 1m 6s
2597:	learn: 0.3540496	total: 7m 11s	remaining: 1m 6s
2598:	learn: 0.3540234	total: 7m 11s	remaining: 1m 6s
2599:	learn: 0.3540109	total: 7m 11s	remaining: 1m 6s
2600:	learn: 0.3539835	total: 7m 11s	remaining: 1m 6s
2601:	learn: 0.3539577	total: 7m 12s	remaining: 1m 6s
2602:	learn: 0.3539256	total: 7m 12s	remaining: 1m 5s
2603:	learn: 0.3539087	total: 7m 12s	remaining: 1m 5s
2604:	learn: 0.3538918	total: 7m 12s	remaining: 1m 5s
2605:	learn: 0.3538656	total: 7m 12s	remaining: 1m 5s
2606:	learn: 0.3538480	total: 7m 13s	remaining: 1m 5s
2607:	learn: 0.3538290	total: 7m 13s	remaining: 1m 5s
2608:	learn: 0.3538038	total: 7m 13s	remaining: 1m 4s
2609:	learn: 0.3537823	total: 7m 13s	remaining: 1m 4s
2610:	learn: 0.3537625	total: 7m 13s	remaining: 1m 4s
2611:	learn: 0.3537474	total: 7m 13s	remaining: 1m 4s
2612:	learn: 0.3537292	total

2748:	learn: 0.3511920	total: 7m 35s	remaining: 41.6s
2749:	learn: 0.3511727	total: 7m 35s	remaining: 41.4s
2750:	learn: 0.3511527	total: 7m 35s	remaining: 41.2s
2751:	learn: 0.3511353	total: 7m 35s	remaining: 41.1s
2752:	learn: 0.3511191	total: 7m 35s	remaining: 40.9s
2753:	learn: 0.3510955	total: 7m 36s	remaining: 40.7s
2754:	learn: 0.3510757	total: 7m 36s	remaining: 40.6s
2755:	learn: 0.3510615	total: 7m 36s	remaining: 40.4s
2756:	learn: 0.3510481	total: 7m 36s	remaining: 40.2s
2757:	learn: 0.3510319	total: 7m 36s	remaining: 40.1s
2758:	learn: 0.3510162	total: 7m 36s	remaining: 39.9s
2759:	learn: 0.3509994	total: 7m 37s	remaining: 39.7s
2760:	learn: 0.3509778	total: 7m 37s	remaining: 39.6s
2761:	learn: 0.3509622	total: 7m 37s	remaining: 39.4s
2762:	learn: 0.3509448	total: 7m 37s	remaining: 39.3s
2763:	learn: 0.3509309	total: 7m 37s	remaining: 39.1s
2764:	learn: 0.3509109	total: 7m 37s	remaining: 38.9s
2765:	learn: 0.3508963	total: 7m 37s	remaining: 38.7s
2766:	learn: 0.3508954	total

2901:	learn: 0.3484966	total: 7m 59s	remaining: 16.2s
2902:	learn: 0.3484832	total: 8m	remaining: 16s
2903:	learn: 0.3484649	total: 8m	remaining: 15.9s
2904:	learn: 0.3484477	total: 8m	remaining: 15.7s
2905:	learn: 0.3484343	total: 8m	remaining: 15.5s
2906:	learn: 0.3484210	total: 8m	remaining: 15.4s
2907:	learn: 0.3484076	total: 8m	remaining: 15.2s
2908:	learn: 0.3484032	total: 8m	remaining: 15s
2909:	learn: 0.3483886	total: 8m 1s	remaining: 14.9s
2910:	learn: 0.3483705	total: 8m 1s	remaining: 14.7s
2911:	learn: 0.3483488	total: 8m 1s	remaining: 14.5s
2912:	learn: 0.3483243	total: 8m 1s	remaining: 14.4s
2913:	learn: 0.3483084	total: 8m 1s	remaining: 14.2s
2914:	learn: 0.3482947	total: 8m 1s	remaining: 14.1s
2915:	learn: 0.3482743	total: 8m 2s	remaining: 13.9s
2916:	learn: 0.3482531	total: 8m 2s	remaining: 13.7s
2917:	learn: 0.3482288	total: 8m 2s	remaining: 13.6s
2918:	learn: 0.3482164	total: 8m 2s	remaining: 13.4s
2919:	learn: 0.3481996	total: 8m 2s	remaining: 13.2s
2920:	learn: 0.34

In [18]:
roc_auc_score(y_test, clf.predict_proba(x_test)[:,1])

0.8381940900523464

In [19]:
pred = clf.predict_proba(test_set)[:,1]

In [20]:
prediction = pd.DataFrame({'Applicant_ID': test['Applicant_ID'], 'default_status': pred})
prediction.head()

,Applicant_ID,default_status
0,Apcnt_1000032,0.244885
1,Apcnt_1000048,0.249336
2,Apcnt_1000052,0.457801
3,Apcnt_1000076,0.745035
4,Apcnt_1000080,0.164954


In [21]:
prediction.to_csv('Submit99.csv', index = False)

In [22]:
clf.fit(train_set, train_label)

0:	learn: 0.6863968	total: 227ms	remaining: 11m 21s
1:	learn: 0.6798448	total: 446ms	remaining: 11m 8s
2:	learn: 0.6738864	total: 693ms	remaining: 11m 31s
3:	learn: 0.6680298	total: 883ms	remaining: 11m 1s
4:	learn: 0.6622227	total: 1.12s	remaining: 11m 11s
5:	learn: 0.6563989	total: 1.5s	remaining: 12m 30s
6:	learn: 0.6509185	total: 1.73s	remaining: 12m 17s
7:	learn: 0.6454043	total: 2s	remaining: 12m 29s
8:	learn: 0.6400655	total: 2.21s	remaining: 12m 15s
9:	learn: 0.6354894	total: 2.49s	remaining: 12m 24s
10:	learn: 0.6302597	total: 2.62s	remaining: 11m 51s
11:	learn: 0.6255260	total: 2.92s	remaining: 12m 5s
12:	learn: 0.6205865	total: 3.14s	remaining: 12m 2s
13:	learn: 0.6159021	total: 3.38s	remaining: 12m 1s
14:	learn: 0.6114261	total: 3.64s	remaining: 12m 3s
15:	learn: 0.6074166	total: 3.78s	remaining: 11m 45s
16:	learn: 0.6030290	total: 4.02s	remaining: 11m 45s
17:	learn: 0.5989813	total: 4.17s	remaining: 11m 30s
18:	learn: 0.5954357	total: 4.44s	remaining: 11m 36s
19:	learn: 0.

155:	learn: 0.4318024	total: 34s	remaining: 10m 19s
156:	learn: 0.4315704	total: 34.2s	remaining: 10m 19s
157:	learn: 0.4313139	total: 34.4s	remaining: 10m 19s
158:	learn: 0.4310563	total: 34.6s	remaining: 10m 18s
159:	learn: 0.4308274	total: 34.8s	remaining: 10m 18s
160:	learn: 0.4305698	total: 35.1s	remaining: 10m 18s
161:	learn: 0.4303480	total: 35.4s	remaining: 10m 19s
162:	learn: 0.4301244	total: 35.5s	remaining: 10m 18s
163:	learn: 0.4298867	total: 35.8s	remaining: 10m 18s
164:	learn: 0.4296428	total: 35.9s	remaining: 10m 17s
165:	learn: 0.4293831	total: 36.2s	remaining: 10m 17s
166:	learn: 0.4291705	total: 36.4s	remaining: 10m 16s
167:	learn: 0.4288966	total: 36.6s	remaining: 10m 16s
168:	learn: 0.4286311	total: 36.7s	remaining: 10m 15s
169:	learn: 0.4284174	total: 36.9s	remaining: 10m 14s
170:	learn: 0.4282143	total: 37.1s	remaining: 10m 14s
171:	learn: 0.4279891	total: 37.3s	remaining: 10m 13s
172:	learn: 0.4277757	total: 37.6s	remaining: 10m 13s
173:	learn: 0.4275821	total: 3

310:	learn: 0.4129076	total: 1m 4s	remaining: 9m 20s
311:	learn: 0.4128433	total: 1m 4s	remaining: 9m 18s
312:	learn: 0.4127833	total: 1m 5s	remaining: 9m 18s
313:	learn: 0.4127326	total: 1m 5s	remaining: 9m 18s
314:	learn: 0.4126641	total: 1m 5s	remaining: 9m 19s
315:	learn: 0.4126087	total: 1m 5s	remaining: 9m 18s
316:	learn: 0.4125553	total: 1m 5s	remaining: 9m 18s
317:	learn: 0.4125019	total: 1m 6s	remaining: 9m 17s
318:	learn: 0.4124452	total: 1m 6s	remaining: 9m 17s
319:	learn: 0.4123856	total: 1m 6s	remaining: 9m 17s
320:	learn: 0.4123241	total: 1m 6s	remaining: 9m 17s
321:	learn: 0.4122591	total: 1m 7s	remaining: 9m 17s
322:	learn: 0.4122064	total: 1m 7s	remaining: 9m 17s
323:	learn: 0.4121632	total: 1m 7s	remaining: 9m 16s
324:	learn: 0.4120983	total: 1m 7s	remaining: 9m 16s
325:	learn: 0.4120496	total: 1m 7s	remaining: 9m 15s
326:	learn: 0.4120113	total: 1m 8s	remaining: 9m 16s
327:	learn: 0.4119610	total: 1m 8s	remaining: 9m 15s
328:	learn: 0.4119210	total: 1m 8s	remaining: 

464:	learn: 0.4064756	total: 1m 35s	remaining: 8m 41s
465:	learn: 0.4064451	total: 1m 35s	remaining: 8m 40s
466:	learn: 0.4064140	total: 1m 36s	remaining: 8m 41s
467:	learn: 0.4063820	total: 1m 36s	remaining: 8m 40s
468:	learn: 0.4063495	total: 1m 36s	remaining: 8m 40s
469:	learn: 0.4063241	total: 1m 36s	remaining: 8m 40s
470:	learn: 0.4062838	total: 1m 36s	remaining: 8m 40s
471:	learn: 0.4062458	total: 1m 37s	remaining: 8m 39s
472:	learn: 0.4062170	total: 1m 37s	remaining: 8m 40s
473:	learn: 0.4061839	total: 1m 37s	remaining: 8m 39s
474:	learn: 0.4061546	total: 1m 37s	remaining: 8m 39s
475:	learn: 0.4061321	total: 1m 37s	remaining: 8m 39s
476:	learn: 0.4061049	total: 1m 38s	remaining: 8m 38s
477:	learn: 0.4060738	total: 1m 38s	remaining: 8m 38s
478:	learn: 0.4060459	total: 1m 38s	remaining: 8m 37s
479:	learn: 0.4060186	total: 1m 38s	remaining: 8m 37s
480:	learn: 0.4059915	total: 1m 38s	remaining: 8m 37s
481:	learn: 0.4059657	total: 1m 39s	remaining: 8m 37s
482:	learn: 0.4059299	total:

618:	learn: 0.4023250	total: 2m 7s	remaining: 8m 10s
619:	learn: 0.4023009	total: 2m 7s	remaining: 8m 10s
620:	learn: 0.4022701	total: 2m 8s	remaining: 8m 10s
621:	learn: 0.4022429	total: 2m 8s	remaining: 8m 10s
622:	learn: 0.4022275	total: 2m 8s	remaining: 8m 10s
623:	learn: 0.4021996	total: 2m 8s	remaining: 8m 10s
624:	learn: 0.4021791	total: 2m 8s	remaining: 8m 9s
625:	learn: 0.4021545	total: 2m 9s	remaining: 8m 9s
626:	learn: 0.4021199	total: 2m 9s	remaining: 8m 9s
627:	learn: 0.4020999	total: 2m 9s	remaining: 8m 9s
628:	learn: 0.4020833	total: 2m 9s	remaining: 8m 9s
629:	learn: 0.4020519	total: 2m 9s	remaining: 8m 8s
630:	learn: 0.4020345	total: 2m 10s	remaining: 8m 8s
631:	learn: 0.4020100	total: 2m 10s	remaining: 8m 8s
632:	learn: 0.4019856	total: 2m 10s	remaining: 8m 8s
633:	learn: 0.4019592	total: 2m 10s	remaining: 8m 8s
634:	learn: 0.4019462	total: 2m 11s	remaining: 8m 7s
635:	learn: 0.4019297	total: 2m 11s	remaining: 8m 8s
636:	learn: 0.4019000	total: 2m 11s	remaining: 8m 8s

...

2283:	learn: 0.3729571	total: 7m 56s	remaining: 2m 29s
2284:	learn: 0.3729443	total: 7m 56s	remaining: 2m 29s
2285:	learn: 0.3729278	total: 7m 57s	remaining: 2m 29s
2286:	learn: 0.3729135	total: 7m 57s	remaining: 2m 28s
2287:	learn: 0.3728965	total: 7m 57s	remaining: 2m 28s
2288:	learn: 0.3728851	total: 7m 57s	remaining: 2m 28s
2289:	learn: 0.3728677	total: 7m 57s	remaining: 2m 28s
2290:	learn: 0.3728496	total: 7m 58s	remaining: 2m 27s
2291:	learn: 0.3728383	total: 7m 58s	remaining: 2m 27s
2292:	learn: 0.3728256	total: 7m 58s	remaining: 2m 27s
2293:	learn: 0.3728101	total: 7m 58s	remaining: 2m 27s
2294:	learn: 0.3727975	total: 7m 59s	remaining: 2m 27s
2295:	learn: 0.3727902	total: 7m 59s	remaining: 2m 26s
2296:	learn: 0.3727711	total: 7m 59s	remaining: 2m 26s
2297:	learn: 0.3727526	total: 7m 59s	remaining: 2m 26s
2298:	learn: 0.3727352	total: 7m 59s	remaining: 2m 26s
2299:	learn: 0.3727257	total: 8m	remaining: 2m 26s
2300:	learn: 0.3727164	total: 8m	remaining: 2m 25s
2301:	learn: 0.372

2436:	learn: 0.3707877	total: 8m 27s	remaining: 1m 57s
2437:	learn: 0.3707693	total: 8m 27s	remaining: 1m 57s
2438:	learn: 0.3707682	total: 8m 28s	remaining: 1m 56s
2439:	learn: 0.3707532	total: 8m 28s	remaining: 1m 56s
2440:	learn: 0.3707415	total: 8m 28s	remaining: 1m 56s
2441:	learn: 0.3707311	total: 8m 28s	remaining: 1m 56s
2442:	learn: 0.3707139	total: 8m 29s	remaining: 1m 56s
2443:	learn: 0.3707046	total: 8m 29s	remaining: 1m 55s
2444:	learn: 0.3706890	total: 8m 29s	remaining: 1m 55s
2445:	learn: 0.3706733	total: 8m 29s	remaining: 1m 55s
2446:	learn: 0.3706570	total: 8m 30s	remaining: 1m 55s
2447:	learn: 0.3706407	total: 8m 30s	remaining: 1m 55s
2448:	learn: 0.3706244	total: 8m 30s	remaining: 1m 54s
2449:	learn: 0.3706233	total: 8m 31s	remaining: 1m 54s
2450:	learn: 0.3706085	total: 8m 31s	remaining: 1m 54s
2451:	learn: 0.3705941	total: 8m 31s	remaining: 1m 54s
2452:	learn: 0.3705772	total: 8m 31s	remaining: 1m 54s
2453:	learn: 0.3705602	total: 8m 32s	remaining: 1m 53s
2454:	lear

2585:	learn: 0.3687421	total: 8m 59s	remaining: 1m 26s
2586:	learn: 0.3687266	total: 9m	remaining: 1m 26s
2587:	learn: 0.3687150	total: 9m	remaining: 1m 26s
2588:	learn: 0.3686998	total: 9m	remaining: 1m 25s
2589:	learn: 0.3686787	total: 9m	remaining: 1m 25s
2590:	learn: 0.3686641	total: 9m	remaining: 1m 25s
2591:	learn: 0.3686464	total: 9m 1s	remaining: 1m 25s
2592:	learn: 0.3686419	total: 9m 1s	remaining: 1m 24s
2593:	learn: 0.3686274	total: 9m 1s	remaining: 1m 24s
2594:	learn: 0.3686089	total: 9m 1s	remaining: 1m 24s
2595:	learn: 0.3685968	total: 9m 2s	remaining: 1m 24s
2596:	learn: 0.3685827	total: 9m 2s	remaining: 1m 24s
2597:	learn: 0.3685723	total: 9m 2s	remaining: 1m 23s
2598:	learn: 0.3685586	total: 9m 2s	remaining: 1m 23s
2599:	learn: 0.3685580	total: 9m 3s	remaining: 1m 23s
2600:	learn: 0.3685572	total: 9m 3s	remaining: 1m 23s
2601:	learn: 0.3685399	total: 9m 3s	remaining: 1m 23s
2602:	learn: 0.3685387	total: 9m 3s	remaining: 1m 22s
2603:	learn: 0.3685225	total: 9m 3s	remain

2738:	learn: 0.3666903	total: 9m 32s	remaining: 54.5s
2739:	learn: 0.3666761	total: 9m 32s	remaining: 54.3s
2740:	learn: 0.3666584	total: 9m 32s	remaining: 54.1s
2741:	learn: 0.3666463	total: 9m 32s	remaining: 53.9s
2742:	learn: 0.3666317	total: 9m 33s	remaining: 53.7s
2743:	learn: 0.3666167	total: 9m 33s	remaining: 53.5s
2744:	learn: 0.3666067	total: 9m 33s	remaining: 53.3s
2745:	learn: 0.3665850	total: 9m 33s	remaining: 53.1s
2746:	learn: 0.3665669	total: 9m 34s	remaining: 52.9s
2747:	learn: 0.3665546	total: 9m 34s	remaining: 52.7s
2748:	learn: 0.3665405	total: 9m 34s	remaining: 52.5s
2749:	learn: 0.3665305	total: 9m 34s	remaining: 52.2s
2750:	learn: 0.3665263	total: 9m 34s	remaining: 52s
2751:	learn: 0.3665066	total: 9m 35s	remaining: 51.8s
2752:	learn: 0.3664902	total: 9m 35s	remaining: 51.6s
2753:	learn: 0.3664764	total: 9m 35s	remaining: 51.4s
2754:	learn: 0.3664612	total: 9m 35s	remaining: 51.2s
2755:	learn: 0.3664469	total: 9m 35s	remaining: 51s
2756:	learn: 0.3664379	total: 9m

2891:	learn: 0.3645202	total: 10m 5s	remaining: 22.6s
2892:	learn: 0.3645082	total: 10m 5s	remaining: 22.4s
2893:	learn: 0.3644931	total: 10m 5s	remaining: 22.2s
2894:	learn: 0.3644821	total: 10m 5s	remaining: 22s
2895:	learn: 0.3644703	total: 10m 5s	remaining: 21.8s
2896:	learn: 0.3644522	total: 10m 6s	remaining: 21.6s
2897:	learn: 0.3644381	total: 10m 6s	remaining: 21.3s
2898:	learn: 0.3644232	total: 10m 6s	remaining: 21.1s
2899:	learn: 0.3644090	total: 10m 6s	remaining: 20.9s
2900:	learn: 0.3643927	total: 10m 6s	remaining: 20.7s
2901:	learn: 0.3643793	total: 10m 7s	remaining: 20.5s
2902:	learn: 0.3643628	total: 10m 7s	remaining: 20.3s
2903:	learn: 0.3643473	total: 10m 7s	remaining: 20.1s
2904:	learn: 0.3643324	total: 10m 7s	remaining: 19.9s
2905:	learn: 0.3643187	total: 10m 7s	remaining: 19.7s
2906:	learn: 0.3643077	total: 10m 8s	remaining: 19.5s
2907:	learn: 0.3642902	total: 10m 8s	remaining: 19.2s
2908:	learn: 0.3642895	total: 10m 8s	remaining: 19s
2909:	learn: 0.3642748	total: 10

In [23]:
pred =clf.predict_proba(test_set)[:,1]

In [24]:
prediction = pd.DataFrame({'Applicant_ID': test['Applicant_ID'], 'default_status': pred})
prediction.head()

,Applicant_ID,default_status
0,Apcnt_1000032,0.323730
1,Apcnt_1000048,0.352258
2,Apcnt_1000052,0.404409
3,Apcnt_1000076,0.725102
4,Apcnt_1000080,0.176438


In [25]:
prediction.to_csv('Submit100.csv', index = False)

In [38]:
from sklearn.model_selection import StratifiedKFold
def crossvaltest(train_set,train_label):
    
    kfold = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 1)
        
    acc, auc = [], []
    for train_index, test_index in kfold.split(train_set, train_label):
        train = train_set.iloc[train_index,:]
        test = train_set.iloc[test_index,:]
        
        labels = train_label[train_index]
        test_labels = train_label[test_index]
        
        clf = cb.CatBoostClassifier(n_estimators = 3000, early_stopping_rounds = 50, learning_rate = 0.01, random_seed =50)
        clf.fit(train, labels)
        
        acc.append(np.mean(clf.predict(test)==np.ravel(test_labels)))
        auc.append(roc_auc_score(test_labels, clf.predict_proba(test)[:,1]))
        pred = clf.predict_proba(test_set)[:,1]
                
    return np.mean(acc), np.mean(auc), pred

In [39]:
model = crossvaltest(train_set, train_label)

0:	learn: 0.6865053	total: 323ms	remaining: 16m 9s
1:	learn: 0.6803076	total: 471ms	remaining: 11m 45s
2:	learn: 0.6742954	total: 739ms	remaining: 12m 18s
3:	learn: 0.6682248	total: 848ms	remaining: 10m 35s
4:	learn: 0.6627067	total: 1.18s	remaining: 11m 48s
5:	learn: 0.6574185	total: 1.3s	remaining: 10m 50s
6:	learn: 0.6519460	total: 1.77s	remaining: 12m 36s
7:	learn: 0.6463011	total: 1.96s	remaining: 12m 11s
8:	learn: 0.6410870	total: 2.2s	remaining: 12m 9s
9:	learn: 0.6358692	total: 2.38s	remaining: 11m 50s
10:	learn: 0.6309213	total: 2.69s	remaining: 12m 10s
11:	learn: 0.6264127	total: 2.85s	remaining: 11m 50s
12:	learn: 0.6217447	total: 3.03s	remaining: 11m 35s
13:	learn: 0.6170158	total: 3.15s	remaining: 11m 12s
14:	learn: 0.6126449	total: 3.33s	remaining: 11m 3s
15:	learn: 0.6084079	total: 3.44s	remaining: 10m 42s
16:	learn: 0.6042307	total: 3.69s	remaining: 10m 47s
17:	learn: 0.6001593	total: 3.81s	remaining: 10m 30s
18:	learn: 0.5966042	total: 3.98s	remaining: 10m 24s
19:	lear

157:	learn: 0.4334228	total: 31.8s	remaining: 9m 31s
158:	learn: 0.4331739	total: 31.9s	remaining: 9m 29s
159:	learn: 0.4329038	total: 32.1s	remaining: 9m 30s
160:	learn: 0.4326841	total: 32.2s	remaining: 9m 28s
161:	learn: 0.4324176	total: 32.4s	remaining: 9m 27s
162:	learn: 0.4322100	total: 32.5s	remaining: 9m 25s
163:	learn: 0.4319901	total: 32.7s	remaining: 9m 25s
164:	learn: 0.4317552	total: 32.9s	remaining: 9m 25s
165:	learn: 0.4314973	total: 33s	remaining: 9m 23s
166:	learn: 0.4312888	total: 33.2s	remaining: 9m 22s
167:	learn: 0.4310664	total: 33.4s	remaining: 9m 22s
168:	learn: 0.4308111	total: 33.5s	remaining: 9m 21s
169:	learn: 0.4305736	total: 33.7s	remaining: 9m 21s
170:	learn: 0.4303290	total: 33.8s	remaining: 9m 20s
171:	learn: 0.4301136	total: 34.1s	remaining: 9m 20s
172:	learn: 0.4298785	total: 34.3s	remaining: 9m 19s
173:	learn: 0.4296535	total: 34.5s	remaining: 9m 20s
174:	learn: 0.4294410	total: 34.6s	remaining: 9m 18s
175:	learn: 0.4292545	total: 34.9s	remaining: 9m

314:	learn: 0.4140148	total: 1m 1s	remaining: 8m 48s
315:	learn: 0.4139633	total: 1m 2s	remaining: 8m 48s
316:	learn: 0.4138874	total: 1m 2s	remaining: 8m 47s
317:	learn: 0.4138315	total: 1m 2s	remaining: 8m 48s
318:	learn: 0.4137832	total: 1m 2s	remaining: 8m 47s
319:	learn: 0.4137237	total: 1m 2s	remaining: 8m 47s
320:	learn: 0.4136760	total: 1m 3s	remaining: 8m 46s
321:	learn: 0.4136103	total: 1m 3s	remaining: 8m 46s
322:	learn: 0.4135410	total: 1m 3s	remaining: 8m 45s
323:	learn: 0.4134921	total: 1m 3s	remaining: 8m 45s
324:	learn: 0.4134212	total: 1m 3s	remaining: 8m 46s
325:	learn: 0.4133703	total: 1m 4s	remaining: 8m 46s
326:	learn: 0.4133208	total: 1m 4s	remaining: 8m 46s
327:	learn: 0.4132827	total: 1m 4s	remaining: 8m 46s
328:	learn: 0.4132370	total: 1m 4s	remaining: 8m 46s
329:	learn: 0.4131800	total: 1m 5s	remaining: 8m 45s
330:	learn: 0.4131159	total: 1m 5s	remaining: 8m 46s
331:	learn: 0.4130507	total: 1m 5s	remaining: 8m 45s
332:	learn: 0.4129903	total: 1m 5s	remaining: 

467:	learn: 0.4072463	total: 1m 33s	remaining: 8m 23s
468:	learn: 0.4072157	total: 1m 33s	remaining: 8m 24s
469:	learn: 0.4071830	total: 1m 33s	remaining: 8m 23s
470:	learn: 0.4071475	total: 1m 33s	remaining: 8m 23s
471:	learn: 0.4070987	total: 1m 33s	remaining: 8m 22s
472:	learn: 0.4070715	total: 1m 34s	remaining: 8m 22s
473:	learn: 0.4070390	total: 1m 34s	remaining: 8m 21s
474:	learn: 0.4070069	total: 1m 34s	remaining: 8m 21s
475:	learn: 0.4069766	total: 1m 34s	remaining: 8m 21s
476:	learn: 0.4069368	total: 1m 34s	remaining: 8m 20s
477:	learn: 0.4069045	total: 1m 34s	remaining: 8m 20s
478:	learn: 0.4068673	total: 1m 35s	remaining: 8m 20s
479:	learn: 0.4068307	total: 1m 35s	remaining: 8m 20s
480:	learn: 0.4067814	total: 1m 35s	remaining: 8m 20s
481:	learn: 0.4067516	total: 1m 35s	remaining: 8m 20s
482:	learn: 0.4067186	total: 1m 36s	remaining: 8m 20s
483:	learn: 0.4066872	total: 1m 36s	remaining: 8m 19s
484:	learn: 0.4066572	total: 1m 36s	remaining: 8m 19s
485:	learn: 0.4066268	total:

621:	learn: 0.4027532	total: 2m 2s	remaining: 7m 46s
622:	learn: 0.4027263	total: 2m 2s	remaining: 7m 46s
623:	learn: 0.4027054	total: 2m 2s	remaining: 7m 46s
624:	learn: 0.4026848	total: 2m 2s	remaining: 7m 45s
625:	learn: 0.4026560	total: 2m 2s	remaining: 7m 45s
626:	learn: 0.4026291	total: 2m 2s	remaining: 7m 45s
627:	learn: 0.4026000	total: 2m 3s	remaining: 7m 44s
628:	learn: 0.4025781	total: 2m 3s	remaining: 7m 44s
629:	learn: 0.4025566	total: 2m 3s	remaining: 7m 44s
630:	learn: 0.4025301	total: 2m 3s	remaining: 7m 44s
631:	learn: 0.4025003	total: 2m 4s	remaining: 7m 44s
632:	learn: 0.4024687	total: 2m 4s	remaining: 7m 44s
633:	learn: 0.4024442	total: 2m 4s	remaining: 7m 44s
634:	learn: 0.4024097	total: 2m 4s	remaining: 7m 44s
635:	learn: 0.4023858	total: 2m 4s	remaining: 7m 44s
636:	learn: 0.4023732	total: 2m 4s	remaining: 7m 43s
637:	learn: 0.4023446	total: 2m 5s	remaining: 7m 43s
638:	learn: 0.4023313	total: 2m 5s	remaining: 7m 43s
639:	learn: 0.4023017	total: 2m 5s	remaining: 

...

2357:	learn: 0.3654179	total: 4m 53s	remaining: 1m 19s
2358:	learn: 0.3654005	total: 4m 53s	remaining: 1m 19s
2359:	learn: 0.3653839	total: 4m 53s	remaining: 1m 19s
2360:	learn: 0.3653681	total: 4m 53s	remaining: 1m 19s
2361:	learn: 0.3653528	total: 4m 53s	remaining: 1m 19s
2362:	learn: 0.3653366	total: 4m 54s	remaining: 1m 19s
2363:	learn: 0.3653153	total: 4m 54s	remaining: 1m 19s
2364:	learn: 0.3653022	total: 4m 54s	remaining: 1m 19s
2365:	learn: 0.3652809	total: 4m 54s	remaining: 1m 18s
2366:	learn: 0.3652548	total: 4m 54s	remaining: 1m 18s
2367:	learn: 0.3652399	total: 4m 54s	remaining: 1m 18s
2368:	learn: 0.3652278	total: 4m 54s	remaining: 1m 18s
2369:	learn: 0.3652112	total: 4m 54s	remaining: 1m 18s
2370:	learn: 0.3651953	total: 4m 54s	remaining: 1m 18s
2371:	learn: 0.3651760	total: 4m 55s	remaining: 1m 18s
2372:	learn: 0.3651553	total: 4m 55s	remaining: 1m 18s
2373:	learn: 0.3651326	total: 4m 55s	remaining: 1m 17s
2374:	learn: 0.3651140	total: 4m 55s	remaining: 1m 17s
2375:	lear

2511:	learn: 0.3626841	total: 5m 12s	remaining: 1m
2512:	learn: 0.3626641	total: 5m 12s	remaining: 1m
2513:	learn: 0.3626539	total: 5m 12s	remaining: 1m
2514:	learn: 0.3626433	total: 5m 13s	remaining: 1m
2515:	learn: 0.3626303	total: 5m 13s	remaining: 1m
2516:	learn: 0.3626191	total: 5m 13s	remaining: 1m
2517:	learn: 0.3626076	total: 5m 13s	remaining: 60s
2518:	learn: 0.3625886	total: 5m 13s	remaining: 59.9s
2519:	learn: 0.3625643	total: 5m 13s	remaining: 59.7s
2520:	learn: 0.3625503	total: 5m 13s	remaining: 59.6s
2521:	learn: 0.3625374	total: 5m 13s	remaining: 59.5s
2522:	learn: 0.3625185	total: 5m 14s	remaining: 59.4s
2523:	learn: 0.3624994	total: 5m 14s	remaining: 59.2s
2524:	learn: 0.3624814	total: 5m 14s	remaining: 59.1s
2525:	learn: 0.3624621	total: 5m 14s	remaining: 59s
2526:	learn: 0.3624513	total: 5m 14s	remaining: 58.9s
2527:	learn: 0.3624301	total: 5m 14s	remaining: 58.7s
2528:	learn: 0.3624130	total: 5m 14s	remaining: 58.6s
2529:	learn: 0.3624122	total: 5m 14s	remaining: 58

2664:	learn: 0.3601789	total: 5m 30s	remaining: 41.5s
2665:	learn: 0.3601597	total: 5m 30s	remaining: 41.4s
2666:	learn: 0.3601451	total: 5m 30s	remaining: 41.3s
2667:	learn: 0.3601318	total: 5m 30s	remaining: 41.1s
2668:	learn: 0.3601205	total: 5m 30s	remaining: 41s
2669:	learn: 0.3601080	total: 5m 30s	remaining: 40.9s
2670:	learn: 0.3600918	total: 5m 30s	remaining: 40.8s
2671:	learn: 0.3600738	total: 5m 31s	remaining: 40.6s
2672:	learn: 0.3600545	total: 5m 31s	remaining: 40.5s
2673:	learn: 0.3600390	total: 5m 31s	remaining: 40.4s
2674:	learn: 0.3600281	total: 5m 31s	remaining: 40.3s
2675:	learn: 0.3600137	total: 5m 31s	remaining: 40.2s
2676:	learn: 0.3599893	total: 5m 31s	remaining: 40s
2677:	learn: 0.3599747	total: 5m 31s	remaining: 39.9s
2678:	learn: 0.3599576	total: 5m 32s	remaining: 39.8s
2679:	learn: 0.3599423	total: 5m 32s	remaining: 39.7s
2680:	learn: 0.3599179	total: 5m 32s	remaining: 39.5s
2681:	learn: 0.3599010	total: 5m 32s	remaining: 39.4s
2682:	learn: 0.3599002	total: 5m

2817:	learn: 0.3577285	total: 5m 48s	remaining: 22.5s
2818:	learn: 0.3577057	total: 5m 49s	remaining: 22.4s
2819:	learn: 0.3576870	total: 5m 49s	remaining: 22.3s
2820:	learn: 0.3576642	total: 5m 49s	remaining: 22.2s
2821:	learn: 0.3576424	total: 5m 49s	remaining: 22s
2822:	learn: 0.3576220	total: 5m 49s	remaining: 21.9s
2823:	learn: 0.3575996	total: 5m 49s	remaining: 21.8s
2824:	learn: 0.3575895	total: 5m 49s	remaining: 21.7s
2825:	learn: 0.3575717	total: 5m 49s	remaining: 21.5s
2826:	learn: 0.3575546	total: 5m 50s	remaining: 21.4s
2827:	learn: 0.3575383	total: 5m 50s	remaining: 21.3s
2828:	learn: 0.3575217	total: 5m 50s	remaining: 21.2s
2829:	learn: 0.3575091	total: 5m 50s	remaining: 21s
2830:	learn: 0.3574914	total: 5m 50s	remaining: 20.9s
2831:	learn: 0.3574758	total: 5m 50s	remaining: 20.8s
2832:	learn: 0.3574625	total: 5m 50s	remaining: 20.7s
2833:	learn: 0.3574459	total: 5m 50s	remaining: 20.5s
2834:	learn: 0.3574453	total: 5m 50s	remaining: 20.4s
2835:	learn: 0.3574362	total: 5m

2973:	learn: 0.3552577	total: 6m 9s	remaining: 3.23s
2974:	learn: 0.3552344	total: 6m 9s	remaining: 3.1s
2975:	learn: 0.3552190	total: 6m 9s	remaining: 2.98s
2976:	learn: 0.3552088	total: 6m 9s	remaining: 2.85s
2977:	learn: 0.3551965	total: 6m 9s	remaining: 2.73s
2978:	learn: 0.3551724	total: 6m 9s	remaining: 2.61s
2979:	learn: 0.3551552	total: 6m 9s	remaining: 2.48s
2980:	learn: 0.3551430	total: 6m 10s	remaining: 2.36s
2981:	learn: 0.3551260	total: 6m 10s	remaining: 2.23s
2982:	learn: 0.3551095	total: 6m 10s	remaining: 2.11s
2983:	learn: 0.3550852	total: 6m 10s	remaining: 1.99s
2984:	learn: 0.3550742	total: 6m 10s	remaining: 1.86s
2985:	learn: 0.3550540	total: 6m 10s	remaining: 1.74s
2986:	learn: 0.3550411	total: 6m 10s	remaining: 1.61s
2987:	learn: 0.3550231	total: 6m 10s	remaining: 1.49s
2988:	learn: 0.3550050	total: 6m 11s	remaining: 1.36s
2989:	learn: 0.3549870	total: 6m 11s	remaining: 1.24s
2990:	learn: 0.3549699	total: 6m 11s	remaining: 1.12s
2991:	learn: 0.3549481	total: 6m 11s

In [40]:
model

(0.8100714285714286,
 0.8409978363664337,
 array([0.30986957, 0.35519241, 0.34412634, ..., 0.29277997, 0.53385546,
        0.19488576]))

In [42]:
model[2][:5]

array([0.30986957, 0.35519241, 0.34412634, 0.73482847, 0.1921783 ])

In [45]:
pred = model[2]

In [46]:
prediction = pd.DataFrame({'Applicant_ID': test['Applicant_ID'], 'default_status': pred})
prediction.head()

,Applicant_ID,default_status
0,Apcnt_1000032,0.309870
1,Apcnt_1000048,0.355192
2,Apcnt_1000052,0.344126
3,Apcnt_1000076,0.734828
4,Apcnt_1000080,0.192178


In [48]:
prediction.to_csv('submit104.csv', index = False)